In [1]:
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall git+https://github.com/The-AI-Alliance/gofannon.git --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.0/345.0 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.0/457.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 4.2 MB/s eta 0

In [2]:
from openai import OpenAI
from google.colab import userdata
import json
# Create an OpenAI client with your deepinfra token and endpoint
openai = OpenAI(
    api_key=userdata.get('deepinfra'),
    base_url="https://api.deepinfra.com/v1/openai",
)

from gofannon.basic_math import Addition, \
  Subtraction, \
  Multiplication, \
  Division, \
  Exponents


tool_list =  [F() for F in [Addition, Subtraction, Multiplication, Division, Exponents]]
tool_map = {f.name: f.fn for f in tool_list}
tool_desc_list = [f.definition for f in tool_list]

{'name': 'ExampleTool', 'description': 'This is an example tool', 'version': '1.0.0'}
Function called with (1, 2) {'key': 'value'}


In [14]:
# here is the user request
messages = [
    {
        "role": "user",
        "content": "What is 12*457?"
    }
]

# let's send the request and print the response
response = openai.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct",
    messages=messages,
    tools=tool_desc_list,
    tool_choice="auto",
)
tool_calls = response.choices[0].message.tool_calls
for tool_call in tool_calls:
    print(tool_call.model_dump())


{'id': 'call_0igNucUlDtwYCqJACHvEKxgM', 'function': {'arguments': '{"num1": 12.0, "num2": 457.0}', 'name': 'multiplication'}, 'type': 'function'}


In [15]:
# extend conversation with assistant's reply
messages.append(response.choices[0].message)

for tool_call in tool_calls:
  function_name = tool_call.function.name
  function_args = json.loads(tool_call.function.arguments)
  function_response = tool_map[function_name](**function_args)

  # extend conversation with function response
  messages.append({
      "tool_call_id": tool_call.id,
      "role": "tool",
      "content": function_response,
  })


# get a new response from the model where it can see the function responses
second_response = openai.chat.completions.create(
  model="meta-llama/Meta-Llama-3.1-70B-Instruct",
  messages=messages,
  tools=tool_desc_list,
  tool_choice="auto",
)

print(second_response.choices[0].message.content)


The answer is 5484.
